<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/spikformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-09-11 07:13:07--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2023-09-11 07:13:07 (29.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
len(text)

1115394

In [4]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [6]:
print(''.join(chars))
vocab_size


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


65

In [7]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}

In [8]:
itos = {i:ch for i,ch in enumerate(chars)}

In [9]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[x] for x in l])

In [10]:
decode = lambda l: "".join([itos[x] for x in l])

In [11]:
encode("hi there")

[46, 47, 1, 58, 46, 43, 56, 43]

In [12]:
decode([46, 47, 1, 58, 46, 43, 56, 43])

'hi there'

In [13]:
import torch

In [14]:
data = torch.tensor(encode(text), dtype = torch.long)

In [15]:
data.shape, type(data)

(torch.Size([1115394]), torch.Tensor)

In [16]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [17]:
train_data = data[:n]
val_data = data[n:]

In [18]:
val_data = data[n:]

In [19]:
block_size = 8

In [20]:
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [21]:
torch.manual_seed(1337)

In [22]:
batch_size = 4

In [23]:
block_size = 8

In [24]:
torch.randint(6, (4,))

tensor([1, 1, 2, 4])

In [25]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [26]:
xb, yb = get_batch('train')

In [27]:
xb

tensor([[57, 43, 60, 43, 52,  1, 63, 43],
        [60, 43, 42,  8,  0, 25, 63,  1],
        [56, 42,  5, 57,  1, 57, 39, 49],
        [43, 57, 58, 63,  6,  1, 58, 46]])

In [28]:
yb

tensor([[43, 60, 43, 52,  1, 63, 43, 39],
        [43, 42,  8,  0, 25, 63,  1, 45],
        [42,  5, 57,  1, 57, 39, 49, 43],
        [57, 58, 63,  6,  1, 58, 46, 47]])

In [29]:
vocab_size

65

In [30]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1137)

In [31]:
device = 'cuda'

In [32]:
dropout = 0.0

In [33]:
!pip install snntorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.1 MB/s eta 0:00:00


In [34]:
import snntorch as snn
from snntorch import surrogate
from snntorch import utils

In [35]:
num_steps = 10 # number of time steps
batch_size = 1
beta = 0.5  # neuron decay rate
spike_grad = surrogate.fast_sigmoid()

In [36]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias = False)
        self.query = nn.Linear(n_embed, head_size, bias = False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)).to(device))
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):

        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v


        return  out

class MulitHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size, dropout=0):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)
        self.lif = snn.Leaky(beta=beta)
        self.inp = snn.Leaky(beta=beta)


    def forward(self, x):
        spk_rec = []
        mem_rec = []

        mem_inp_rec = []
        spk_inp_rec = []

        mem = self.lif.init_leaky()
        mem_inp = self.inp.init_leaky()
        for step in range(num_steps):
            spk_inp, mem_inp = self.inp(x, mem_inp)
            x =  torch.cat([head(spk_inp) for head in self.heads], dim=-1)
            out = self.dropout(self.proj(x))
            spk, mem = self.lif(out, mem)
            spk_rec.append(spk)
            mem_rec.append(mem)
        return torch.stack(spk_rec, dim=0).mean(dim=0)



class FeedForward(nn.Module):
    def __init__(self, n_embed, dropout=0):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
         nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_head, n_cols):
        super().__init__()
        self.sa_head= MulitHeadAttention(n_head, n_embed//n_head)
        self.ffw=  FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x + self.ffw(self.ln2(x))
        return x


class Transformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embed).to(device)
        self.position_embedding_table = nn.Embedding(block_size, n_embed).to(device)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head, n_cols=4) for _ in range(n_layer)])
        self.ln = nn.LayerNorm(n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)


    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx).to(device)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = token_emb + pos_emb
        x = self.blocks(x)
        x = self.ln(x)
        logits = self.lm_head(x)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokes):
        for _ in range(max_new_tokes):
            idx_cond = idx[:, -block_size:].to(device)
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [37]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd =n_embed = 32
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [38]:
model = Transformer()
model  = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [39]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y=Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [43]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    if len(yb) != batch_size:
        continue
    xb = xb.to(device)
    yb= yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.4253, val loss 2.4322


KeyboardInterrupt: ignored

In [44]:
model.eval()
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))



BUCor, weal moroum her.
Al yo EDeathin lis br; fo, his hele;
O chinow mes, an stru heringonrce msh come thetu; 'gu sereromy soom s?

JGETIEDY Motuthelly kis bruo'sacaceng; n,
Andend thile d withitidge


In [45]:
n_head

4

In [46]:
n_layer

4

In [47]:
batch_size=64

In [48]:
block_size=32

In [49]:
eval_interval=500

In [50]:
n_embed = 32
n_layer = 4

In [51]:
learning_rate=3e-4

In [52]:
n_head = 6

In [53]:
dropout=0.2

In [55]:
# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 32
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

In [56]:
model = Transformer()
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [57]:
len(text.split(' '))

169893

In [58]:
len(text)

1115394

In [59]:
chars = sorted(list(set(text.split(' '))))
vocab_size = len(chars)

In [60]:
chars[2]

'\nWas'

In [61]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: " ".join([itos[x] for x in l])

In [62]:
data = torch.tensor(encode(text.split(' ')), dtype = torch.long)

In [63]:
data[:10]

tensor([ 1455,   957, 39874, 29614,  5949, 16628, 18572, 24432, 34050, 34057])

In [64]:
decode(encode(text.split(' ')[:10]))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst'

In [65]:
text.split(' ')[:10]

['First',
 'Citizen:\nBefore',
 'we',
 'proceed',
 'any',
 'further,',
 'hear',
 'me',
 'speak.\n\nAll:\nSpeak,',
 'speak.\n\nFirst']

In [66]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [67]:
model = Transformer()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [68]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb  = yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 10.7973, val loss 10.8028
step 100: train loss 9.8105, val loss 9.8503
step 200: train loss 8.9541, val loss 9.0816


KeyboardInterrupt: ignored

In [69]:
context = torch.tensor([encode("thou art kneel before king".split(' '))], dtype = torch.long).to(device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


thou art kneel before king banish belly's life
That complaint
Intended upon
me.

Clown:
What, conclude, before:
The a cloudy.

SEBASTIAN:
Foul violence ANNE:
Despiteful the
fairer thee? die,
Until lie
His wise,
And edge Yes, less.

Nurse:
No tumbling power,
Provided intend horn it!
I'll calm soldier
Even through.
Now, legs
Dared thousand touch man;
Fly presently.' coronation.
In by Servant:
Will't springs, chairs dower;
For gone,
To fellow,
What's princely
exercises grown
from it; grace son.

DUKE mans guests are hate express won boar, Send your
command MARGARET:
Hover imprison'd daughter:
The touch
But clean:
You BOLINGBROKE:
O best:
And, bear?
For do;
To a not
Like care hare a Harry: so blame is.

KATHARINA:
Pardon, consistory,
My flame God's I sunder injury Edward's year; children, proud.

Second cannot
Believe I desired death York,
Or pheasant: sound County thus?
This Venus ditch,
And teeth,--
And between Time's MARGARET:
Urge fly you.

ROMEO:
Gentle but liege; clothes! herself
But

In [70]:
context = torch.tensor([encode("thou art kneel before king".split(' '))], dtype = torch.long).to(device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


thou art kneel before king bears, pray, seals riches approaches: perish! bravely particularly against hatches: sail!

BENVOLIO:
Two, beguiled
The armies?

Messenger:
Within it.

QUEEN But, he
moves seconded; shall adversary,
And is.
Ten desires and pieces, king.

OXFORD:
Call run
A spurn their bondmaid forget'st. Citizen:
Think gust YORK:
Either left more
thankful pretend too!

GLOUCESTER:
Marry, villain few people's but doit, she Watchman:
Who you,
But pleasure liberty,
My the march a vulgars circumspect.
Boy!

Page:
My murder, that there not: usurp'd.

KING affection, logs:
Call stage hand,
And law,
To wives lavish, apology?

BENVOLIO:
The think, best.

KING true--
But pace weal; bound? looks
My brothers
More seem, our highness,
My dream;
From along.

AUTOLYCUS:
Now, worthy any?

LUCIO:
Some myself
I bear revenged.

BAPTISTA:
What, tapers hate!
Too sow'd,
and II:
And him.

First thus: showers libels men's north,
When liberty a peaces.

First corns object ghost: certainty. Burgundy.



In [71]:
context = torch.tensor([encode("Hermione".split(' '))], dtype = torch.long).to(device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


Hermione arms:
Their mistress.

LEONTES:
Satisfy!
The rough, Vere,
Was when recompense:
My duke.

LUCIO:
O, more,
That again!
I all?
Shall appear, enemies.

YORK:
Then briefly, fond love?

JULIET:
But ANNE:
Arise, Catesby:
You ground,
May Earl so. beams,
Her beloved,
In and subjects? time, people.

Third be not an pleasure pleasing minstrel.

First course,
And speak
My country,
How open smallest thou rope-tricks. palace substance, convey'd
Unto Rosaline:
And men end; glean after is, fill
Ithaca was
To GAUNT:
Thy the bigger; on,
And rather visiting shepherd: days:
O, lives, heaven
Would Rosaline!
How I
say, boy,
If soul,
Was pertinent
Than strike!
'Tis provincial. death both.

JULIET:
Amen!

Nurse:
What?

JULIET:
Well, veil them.

BRUTUS:
Go do.

CORIOLANUS:
Thou eye
Red devil. traitor:
Off stretch-mouthed veil'd VI:
That's shortly,
I'll again.
Evermore absence exile defy brother, vastidity channel maidenheads rough, foundation,
And forgot childish up: bosom,
When meteor probable hold h

In [72]:
with open('dialogs.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [73]:
chars = sorted(list(set(text.split(' '))))
vocab_size = len(chars)

In [74]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: " ".join([itos[x] for x in l])

In [75]:
data = torch.tensor(encode(text.split(' ')), dtype = torch.long)

In [76]:
decode(encode(text.split(' ')[:10]))

"hi, how are you doing?\ti'm fine. how about yourself?\ni'm fine."

In [77]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [80]:
model = Transformer()
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [81]:
context = torch.tensor([encode("busy".split(' '))], dtype = torch.long).to(device)
print(decode(model.generate(context, max_new_tokes=200)[0].tolist()))


busy door.
but hunch.
i about?
so playing.
you mean?	while thing job.	yes, heaven you.	what artist noticed.	well, payments street.
i plane.
i pets?	no they?
those cool.
well, lonely. sausage, what?
doing what?	of choose bank.	what sale?	yes, tag yet usual.
well, jet nights.	and paint?
did math.	well, arms.
look mean?	if hour.
they rob won.	i artist.
he's there.	good offering that.
i'll converter?
did full.
i'm tires buy?	i glad!"	what bath me.
and doctor.
so, door.
but light parts.
it dirty, shoes?	my took pretty.
she's say?	the bill class?	that go?
have cars.
eleven car.	me, problems.
everyone random, miss up.	i'll gun.	everyone eat spot.
that games.	that one.
yes, first!	i've cares stains, steak.	i find. strange.	that's eyes?	i helps nice.	it's parents.
i tickets?	yes, many?
how wait.	we school.	i washings, fun?
how right.	like mail me!
i cool me.
yes, boy.
your show.
i old.	well, well, intend friends.
of aisle.
you're exciting.
to already!
i'll gone.
what solution.
i $400.	i'll joki

In [94]:

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 6.6048, val loss 7.7272
step 100: train loss 6.5612, val loss 7.7212
step 200: train loss 6.5142, val loss 7.7205
step 300: train loss 6.4497, val loss 7.7226
step 400: train loss 6.4018, val loss 7.7499
step 500: train loss 6.3349, val loss 7.7660
step 600: train loss 6.2516, val loss 7.7628
step 700: train loss 6.2162, val loss 7.7805


KeyboardInterrupt: ignored

In [97]:
model.eval()
context = torch.tensor([encode("hi, how".split(' '))], dtype = torch.long).to(device)
print(decode(model.generate(context, max_new_tokes=50)[0].tolist()))


hi, how should you.	if going to thatcigarettes every this care. the california.
it's i those day next about button where i wish my numbers.
first a died.	i'm down, he usually and me?
okay. if a door.
but calculator?
do happy.	congratulations the first need water too problem?	they how i right.
i will you like to see about everywhere.
paper pounds?	my
